In [1]:
class Knoten:
    __slots__ = ['kanten', 'index', 'szkindex', 'besucht']

    def __init__(self, *kanten):
        self.kanten = kanten  # Liste der Namen der Knoten zu denen dieser Knoten führt
        self.index = 0  # Der Index dieses Knotens im Graphen. Wird im Verlauf des Algorithmus gesetzt
        self.szkindex = 0  # Der Knoten mit dem niedrigsten Index in der aktuellen SZK. Wird ebenfalls im Verlauf gesetzt
        self.besucht = False  # dieser Switch-Wert wechselt für alle Knoten im Graph bei jedem Aufruf von `tarjan(graph)`


"""
    "!a" -> "!b"
    "b" -> "a"
    "a" -> "b"
    "!b" -> "!a"
    "a" -> "!b"
    "b" -> "!a"
    "!a" -> "!c"
    "c" -> "a"
    "!d" -> "!e"
    "e" -> "d"
    "d" -> "e"
    "!e" -> "!d"
    "d" -> "!e"
    "e" -> "!d"
    "!d" -> "!f"
    "f" -> "d"
"""
graph = {
    '!a': Knoten('!b', '!c'),
    'b': Knoten('a', '!a'),
    'a': Knoten('b', '!b'),
    '!b': Knoten('!a'),
    'c': Knoten('a'),
    '!d': Knoten('!e', '!f'),
    'e': Knoten('d', '!d'),
    '!e': Knoten('!d'),
    'd': Knoten('!e', 'e'),
    'f': Knoten('d'),
    '!c': Knoten(),
    '!f': Knoten(),
}


def tarjan(graph):
    if not graph:
        return

    knotenzähler = 0
    pfad, schnellzugriff = [], set()
    besucht = not next(iter(graph.values())).besucht  # Gegenteil der .besucht-Attribute der Knoten im Graph

    def besuche(knotenname,
                aufruflevel=0):  # aufruflevel wird hier nur fürs prettyprinting, nicht für den Algorithmus benötigt
        nonlocal knotenzähler

        knoten = graph[knotenname]
        if knoten.besucht == besucht:
            return

        # Diesen Knoten besuchen
        knoten.index = knotenzähler
        knoten.szkindex = knotenzähler
        knotenzähler += 1
        pfad.append(knotenname);
        schnellzugriff.add(knotenname)
        knoten.besucht = besucht
        prettyprint('initialisiert', knotenname, knoten, aufruflevel)

        # Nachbarknoten besuchen
        for kante in knoten.kanten:
            nächster = graph[kante]
            if nächster.besucht != besucht:
                besuche(kante, aufruflevel + 1)
                knoten.szkindex = min(knoten.szkindex, nächster.szkindex)
            else:
                prettyprint('bereits besucht', knotenname, knoten, aufruflevel, kante=kante)
                if kante in schnellzugriff:
                    knoten.szkindex = min(knoten.szkindex, nächster.index)
        prettyprint('alle kanten besucht', knotenname, knoten, aufruflevel)

        # SZKs ausgeben
        if knoten.szkindex == knoten.index:
            szk = []
            while True:
                pfadknotenname = pfad.pop();
                schnellzugriff.remove(pfadknotenname)
                szk.append(pfadknotenname)
                if pfadknotenname == knotenname:
                    break
            prettyprint('szk gefunden', knotenname, knoten, aufruflevel, szk=szk)

    # Algorithmus starten
    for knotenname in graph:
        besuche(knotenname)


# Diese Funktion wird hier nur verwendet um den Verlauf des Algorithmus zu visualisieren. Der Algorithmus ist davon unabhängig.
def prettyprint(ereignis, knotenname, knoten, aufruflevel, kante=None, szk=None):
    einrückung = aufruflevel * '   '
    sprecher = f"{einrückung}{knotenname}"

    if ereignis == 'initialisiert':
        if knoten.kanten:
            kantenstring = ', '.join(knoten.kanten)
            print(f"{sprecher}: Initialisiert. Besuche nun {kantenstring}")
        else:
            print(f"{sprecher}: Initialisiert. Keine Kanten")

    elif ereignis == 'bereits besucht':
        print(f"{sprecher}: {kante} bereits besucht")

    elif ereignis == 'alle kanten besucht':
        if knoten.kanten:
            print(f"{sprecher}: Alle Kanten besucht")

    elif ereignis == 'szk gefunden':
        if len(szk) > 1:  # Wir sind hier nur an SZKs interessiert die mehr als einen Knoten enthalten
            szk.reverse()
            szk.append(szk[0])
            szk = ' -> '.join(szk)
            print(
                f'{sprecher}: SZK gefunden!\n\n'
                f'{einrückung}   {szk}\n'
            )


# Aufruf des Algorithmus
tarjan(graph)

"""
"""
graph2 = {
    'a': Knoten('b', '!c', '!e', 'e'),
    '!b': Knoten('!a', 'a'),
    '!a': Knoten('!b', 'b'),
    'b': Knoten('a'),
    'c': Knoten('!a', '!f'),
    'd': Knoten('e', 'f'),
    '!e': Knoten('!d', 'd', '!a'),
    '!d': Knoten('!e', 'e'),
    'e': Knoten('d', 'a'),
    '!f': Knoten('!d', 'c'),
    'f': Knoten('!c'),
    '!c': Knoten('f'),
}
tarjan(graph2)


!a: Initialisiert. Besuche nun !b, !c
   !b: Initialisiert. Besuche nun !a
   !b: !a bereits besucht
   !b: Alle Kanten besucht
   !c: Initialisiert. Keine Kanten
!a: Alle Kanten besucht
!a: SZK gefunden!

   !a -> !b -> !a

b: Initialisiert. Besuche nun a, !a
   a: Initialisiert. Besuche nun b, !b
   a: b bereits besucht
   a: !b bereits besucht
   a: Alle Kanten besucht
b: !a bereits besucht
b: Alle Kanten besucht
b: SZK gefunden!

   b -> a -> b

c: Initialisiert. Besuche nun a
c: a bereits besucht
c: Alle Kanten besucht
!d: Initialisiert. Besuche nun !e, !f
   !e: Initialisiert. Besuche nun !d
   !e: !d bereits besucht
   !e: Alle Kanten besucht
   !f: Initialisiert. Keine Kanten
!d: Alle Kanten besucht
!d: SZK gefunden!

   !d -> !e -> !d

e: Initialisiert. Besuche nun d, !d
   d: Initialisiert. Besuche nun !e, e
   d: !e bereits besucht
   d: e bereits besucht
   d: Alle Kanten besucht
e: !d bereits besucht
e: Alle Kanten besucht
e: SZK gefunden!

   e -> d -> e

f: Initialisiert